In [34]:
from data import countries
import pandas as pd
import numpy as np
import scipy.stats as stats

df = pd.DataFrame(countries)
df['percapita(000)'] = df['gdp(billions)']/df['population(millions)']
for col in df.select_dtypes(include=[np.number]).columns:
    skew = stats.skew(df[col].dropna())
    kurt = stats.kurtosis(df[col].dropna())
    print(f"{col}: skew = {skew:.2f}, kurtosis = {kurt:.2f}")

population(millions): skew = 4.64, kurtosis = 21.32
gdp(billions): skew = 5.16, kurtosis = 27.66
happiness_index: skew = -1.36, kurtosis = 1.37
v-dem_idx: skew = -0.45, kurtosis = -1.25
gini_coef: skew = 1.13, kurtosis = 1.53
percapita(000): skew = 0.99, kurtosis = 0.54


In [35]:
# filter and log-transform
df = df[(df['percapita(000)'] > 0) & (df['gdp(billions)'] > 0)].copy()

df['log_percapita(000)'] = np.log(df['percapita(000)'])
df['log_gdp(billions)'] = np.log(df['gdp(billions)'])
# happiness_index and v-dem_idx can stay untransformed if they’re already bounded and symmetric

In [36]:
# fit the log-log Model
from sklearn.linear_model import LinearRegression

X = df[['log_gdp(billions)', 'happiness_index', 'v-dem_idx']]
y = df['log_percapita(000)']

model = LinearRegression()
model.fit(X, y)

print("Intercept:", model.intercept_)
for name, coef in zip(X.columns, model.coef_):
    print(f"{name}: {coef:.4f}")

Intercept: -3.083614915288402
log_gdp(billions): 0.1796
happiness_index: 0.6798
v-dem_idx: 1.4436


In [37]:
# run the model with statsmodels.formula.api
import statsmodels.formula.api as smf

# Make sure your DataFrame is ready
# df should already have: log_percapita, log_gdp, happiness_index, v_dem_idx
formula = 'Q("log_percapita(000)")~Q("log_gdp(billions)")+happiness_index+Q("v-dem_idx")'
model_smf = smf.ols(formula, data=df).fit()
print(model_smf.summary())

                               OLS Regression Results                              
Dep. Variable:     Q("log_percapita(000)")   R-squared:                       0.732
Model:                                 OLS   Adj. R-squared:                  0.718
Method:                      Least Squares   F-statistic:                     52.85
Date:                     Wed, 26 Nov 2025   Prob (F-statistic):           1.35e-16
Time:                             02:48:06   Log-Likelihood:                -72.500
No. Observations:                       62   AIC:                             153.0
Df Residuals:                           58   BIC:                             161.5
Df Model:                                3                                         
Covariance Type:                 nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

In [38]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

X = sm.add_constant(df[['log_gdp(billions)', 'happiness_index', 'v-dem_idx']])
vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

            Variable        VIF
0              const  26.396414
1  log_gdp(billions)   1.218056
2    happiness_index   1.674808
3          v-dem_idx   1.422622


In [39]:
from sklearn.linear_model import LinearRegression

X = df[['log_gdp(billions)', 'happiness_index', 'v-dem_idx']]
y = df['log_percapita(000)']

log_reg = LinearRegression()
log_reg.fit(X, y)

df['log_percapita_pred'] = log_reg.predict(X)
df['percapita_pred'] = np.exp(df['log_percapita_pred'])

In [47]:
df['error_pct'] = 100 * (df['percapita_pred'] - df['percapita(000)']) / df['percapita(000)']

results_table = df[['name', 'percapita(000)', 'percapita_pred', 'error_pct']].copy()
results_table.columns = ['name', 'Actual (000)', 'Predicted (000)', 'Error (%)']
results_table = results_table.sort_values('Error (%)', key=abs, ascending=False)

# Display top 10 largest errors
results_table.head(10).round(4)

,name,Actual (000),Predicted (000),Error (%)
56,congo,0.6932,5.0053,622.0047
45,Nepal,1.5421,7.1226,361.8777
10,brazil,10.6103,36.2421,241.5736
57,pakistan,1.6099,5.0143,211.4752
44,vietnam,4.6667,12.3440,164.5152
50,nigeria,2.4622,6.2964,155.7254
42,guatamala,6.4706,16.1075,148.9335
16,indonesia,5.0704,11.9999,136.6641
31,Costarica,20.6000,46.7736,127.0565
49,gambia,0.9643,2.0045,107.8756


In [49]:
from sklearn.metrics import r2_score

r2_sklearn = r2_score(y, df['log_percapita_pred'])
print(f"Sklearn R² (log-log space): {r2_sklearn:.4f}")
print(f"Statsmodels R²: {model_smf.rsquared:.4f}")

Sklearn R² (log-log space): 0.7322
Statsmodels R²: 0.7322


In [51]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(df['percapita(000)'], df['percapita_pred'])
mse = mean_squared_error(df['percapita(000)'], df['percapita_pred'])
rmse = np.sqrt(mse)

mean_actual = df['percapita(000)'].mean()

mae_pct = 100 * mae / mean_actual
rmse_pct = 100 * rmse / mean_actual

print(f"MAE:  {mae:.2f} (≈ {mae_pct:.2f}% of average per capita)")
print(f"MSE:  {mse:.2f}")
print(f"RMSE: {rmse:.2f} (≈ {rmse_pct:.2f}% of average per capita)")

MAE:  14.09 (≈ 38.02% of average per capita)
MSE:  601.54
RMSE: 24.53 (≈ 66.18% of average per capita)
